In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import json
import sys
import urllib
import requests
import seaborn as sn
import matplotlib.pyplot as plt
# import impyute as impy
sys.setrecursionlimit(100000)

In [2]:
df = pd.read_csv("CaseCompetitionData2021.csv")
df = df.replace('?', np.NaN)

In [3]:
print(df['customer_gender'].value_counts())
df1 = df.replace(to_replace = {"U": 0.5754726985770696, "M": 1, "F": 0})
# df1['customer_gender'].value_counts()
df1['purchase_price'] = df1['purchase_price'].replace(to_replace = {np.NaN: "15001 - 20000"})
df1['customer_age'] = df1['customer_age'].replace(to_replace = {np.NaN: "31 - 40"})

M    184500
F    136106
U     35268
Name: customer_gender, dtype: int64


In [4]:
# df1['customer_distance_to_dealer'].sort_values(ascending=False)

In [5]:
ptr = df1['customer_distance_to_dealer']
lam = lambda x: np.random.exponential(1 / 15.7350160666884)
df1['customer_distance_to_dealer'] = df1['customer_distance_to_dealer'].where(df1['customer_distance_to_dealer'].notna(), other=lam)
df1['customer_distance_to_dealer'] = df1['customer_distance_to_dealer'].apply(float)

# for i in range(len(ptr)):
#     if ptr[i] == np.NaN:
#         print(i)
#         ptr.set_value(i, int(np.random.exponential(1 / 15.7350160666884)))

In [6]:
df1.isna().sum()

insert_num                          0
purchase_make                       0
purchase_model                      0
purchase_vehicle_year               0
purchase_price                      0
trade_in                            0
vehicle_financing                   0
customer_age                        0
customer_income                 45998
customer_gender                     0
customer_previous_purchase          0
customer_distance_to_dealer         0
post_purchase_satisfaction     354676
vehicle_warranty_used               0
subsequent_purchases                0
dtype: int64

In [7]:
print(df1['customer_income'].value_counts())
# print(df1['purchase_price'].value_counts())
print(df1['customer_age'].value_counts())

20001 - 40000      80855
40001 - 60000      77023
60001 - 80000      42556
0 - 20000          32653
80001 - 100000     29633
100001 - 120000    15819
200001+            10906
140001 - 160000     7566
120001 - 140000     7459
160001 - 180000     3755
180001 - 200000     1651
Name: customer_income, dtype: int64
31 - 40     89410
41 - 50     84604
21 - 30     83004
51 - 60     54801
61 - 70     26104
71 - 80      8182
0 - 20       6133
81 - 90      2228
101+         1251
91 - 100      157
Name: customer_age, dtype: int64


In [8]:
dic =  {"31 - 40":     35,
        "41 - 50":     45,
        "21 - 30":     25,
        "51 - 60":     55,
        "61 - 70":     65,
        "71 - 80":     75,
        "0 - 20":      18,
        "81 - 90":     85,
        "101+":        105,
        "91 - 100":    95}
def f(x):
    if type(x) == float:
        return x
    if x[-1] == '+':
        return float(x[:-1]) * 1.1
    sp = x.split(" - ")
    try:
        return (float(sp[0]) + float(sp[1])) / 2
    except:
        print(x)

df1[['purchase_price', 'customer_age', 'customer_income']] = df1[['purchase_price', 'customer_age', 'customer_income']].applymap(f)

In [9]:
df = df1.drop(labels=['insert_num', 'post_purchase_satisfaction'], axis=1)
df = df[df['customer_income'].notna()]
df_onehot = df['purchase_make'].values

In [10]:
enc = OneHotEncoder()
df_onehot = enc.fit_transform(df_onehot.reshape(len(df_onehot), 1))

In [11]:
df_onehot.toarray().shape

(309876, 40)

In [12]:
np.save("one_hot.txt", df_onehot.toarray())
df.to_csv("quant.csv")

In [ ]:
np.concatenate((df_onehot.toarray(), df.values), axis=1).shape

In [ ]:
enc.get_feature_names()
df_onehot = pd.DataFrame(df_onehot, columns=enc.get_feature_names())
df_onehot.head()

In [ ]:
df1.info()   # 355,8794       # for one hot: purchase_make, purchase_price, customer_age, customer_income

In [ ]:
makers = df.groupby("purchase_make")
# makers.sum().sort_values(by=["subsequent_purchases"])["subsequent_purchases"] * makers.mean().sort_values(by=["subsequent_purchases"]) 
df.columns

In [ ]:
corrMatrix = df[['trade_in', 'vehicle_financing', 'purchase_vehicle_year',
                'customer_age', 'customer_income', 'post_purchase_satisfaction',
                'vehicle_warranty_used', 'customer_previous_purchase','subsequent_purchases']].corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:

def get_bodry(make, model, year):
    where = urllib.parse.quote_plus("""
    {
        "Make": "BMW",
        "Model": "3 Series",
        "Year": 2010
    }
    """)
    url = 'https://parseapi.back4app.com/classes/Carmodels_Car_Model_List?limit=10&order=Category&keys=Model,Category&where=%s' % where
    headers = {
        'X-Parse-Application-Id': 'rkypmU4bqzqB1bKeDvI2kfn3xaBmL8u6AkEzy0Hp', # This is your app's application id
        'X-Parse-REST-API-Key': 'iBez9YtHQLKFidNNQv9BxbPytNrSRmGzsUpIUea6' # This is your app's REST API key
    }
    data = json.loads(requests.get(url, headers=headers).content.decode('utf-8')) # Here you have the data that you need
    print(json.dumps(data, indent=2))

In [ ]:
df.groupby("customer_age").mean()[["subsequent_purchases", "customer_previous_purchase"]].plot()

In [ ]:
np.random.exponential(15.7350160666884)